In [ ]:
#  сборщик мусора
import gc
# системные команды
import sys
#ошибки
import warnings
# joblib- это набор инструментов для облегчения конвейерной обработки в Python . В частности:
# прозрачное кеширование функций на диске и ленивая переоценка (шаблон Memoize
# простые простые параллельные вычисления
from joblib import Parallel, delayed
# Пути к объектно-ориентированной файловой системе
from pathlib import Path
#  виджеты, представляют собой интерактивные HTML-виджеты для записных книжек Jupyter и ядра IPython.
import ipywidgets as widgets

# стандартные
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# TensorFlow — открытая программная библиотека для машинного обучения, разработанная компанией Google для решения задач построения и тренировки нейронной сети 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.experimental.preprocessing import StringLookup

In [ ]:
plt.style.use('ggplot')
warnings.simplefilter("ignore")

In [ ]:
# Helper function to unpack json found in daily data
# вспом функция для распаковки ежедневных данных
def unpack_json(json_str):
    return pd.DataFrame() if pd.isna(json_str) else pd.read_json(json_str)
# isna - проверка на пустое значение

In [ ]:
#  функция паралельных вычислений
def unpack_data(data, dfs=None, n_jobs=-1):
    if dfs is not None:
        data = data.loc[:, dfs]
    unnested_dfs = {}
    for name, column in data.iteritems():
        daily_dfs = Parallel(n_jobs=n_jobs)(
            delayed(unpack_json)(item) for date, item in column.iteritems())
        df = pd.concat(daily_dfs)
        unnested_dfs[name] = df
    return unnested_dfs

In [ ]:
data_dir = Path('../input/mlb-player-digital-engagement-forecasting/')

df_names = ['seasons', 'teams', 'players', 'awards']
#  читаем файлы создаем ДФ по имени файла
for name in df_names:
    globals()[name] = pd.read_csv(data_dir / f"{name}.csv")

In [ ]:
# Итоговые данные 
def sumtab(df):
    print(f'Data Shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Dtypes'])
    summary['Null'] = df.isnull().sum().values
    summary['Uniques'] = df.nunique().values
    #summary['First_values'] = df.loc[0].values
    #summary['Second_values'] = df.loc[1].values
    #summary['Third_values'] = df.loc[2].values
    
    return summary

In [ ]:
# my_player = 547989
# targets_player = targets[targets.playerId == my_player]


# scores = scores.rename(columns={'gameDate': 'date'})
# # scores['date'] = pd.PeriodIndex(scores.date, freq='D')
# # scores = scores.set_index('date')


#sns.scatterplot(data=targets_player['target4'])

# Name of target columns
targets = ["target1", "target2", "target3", "target4"]

#train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')

In [ ]:
from sklearn.preprocessing import LabelEncoder
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)
    
#df_coding_columns = ['primaryPositionName']
#players = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(players)

In [ ]:
#data_dir_my = Path('../input/mlb-players-stat/')
df_target_stats = pd.read_csv('../input/mbl-players-stat/player_target_stats.csv')

#df_target_stats = pd.read_csv(data_dir_my / "player_target_stats.csv")

# EDA

###  Общие вопросы по  Датасет
- Какие атрибуты
- какие типы атрибутов
- Есть ли  Null значения
- Масштаб атрибутов

### Players

In [ ]:
players.info()

In [ ]:
players.head()

In [ ]:
# filter players 
players = players[players.playerForTestSetAndFuturePreds == True]

In [ ]:
players.info()

In [ ]:
players_col = players.columns
for col in players_col:
    print()
    print(col)
    print(players[col].value_counts())

In [ ]:
players['birthCountry'].hist()

In [ ]:
df_target_stats = df_target_stats.astype({'playerId':'int64'})  
df_target_stats.info()

In [ ]:
players_stat = players.copy()

In [ ]:
players_stat.info()

In [ ]:
scores_cols = df_target_stats.columns
players_stat= players_stat.merge(df_target_stats[scores_cols], on=['playerId'], how='left')

In [ ]:
players_stat

In [ ]:
time_col = ['mlbDebutDate','DOB']
for col in time_col :
    players_stat[col] = pd.to_datetime(players_stat[col]).values.astype('datetime64[D]')

In [ ]:
from datetime import datetime
from datetime import date

In [ ]:
players_stat['old'] = players_stat['DOB']
players_stat['old'] = pd.to_datetime(players_stat['old'])
#players_stat['old'] = players_stat['old'].append(lambda x: x.year)

In [ ]:
players_stat['year'] = pd.DatetimeIndex(players_stat['old']).year
players_stat['Debut_year'] = pd.DatetimeIndex(players_stat['mlbDebutDate']).year

In [ ]:
players_stat

In [ ]:
today = date.today()
print(today.year)
type(today.year)

In [ ]:
players_stat['year_old'] = players_stat['year'].apply(lambda x:  today.year - x)
players_stat['Debut_old'] = players_stat['Debut_year'].apply(lambda x:  today.year - x)

In [ ]:
players_stat.fillna(0, inplace=True)

In [ ]:
players_stat

In [ ]:
players_stat['Debut_old'].hist()

In [ ]:
players_stat['year_old'].hist()

In [ ]:
players_stat.info()

In [ ]:
col_obj = players_stat.dtypes[players_stat.dtypes == object].index
col_obj

In [ ]:
for col in col_obj:
    print()
    print(col)
    print(players_stat[col].value_counts())

# for i in col_obj :
#     print(i ,players_stat[i].unique() )


In [ ]:
df_coding_columns = ['birthCity','birthCountry']

In [ ]:
df_coding_columns

In [ ]:
#df_coding_columns = ['birthCity','birthStateProvince','birthCountry']
players_stat_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(players_stat)
#players_stat_incod = players_stat.copy()

In [ ]:
players_stat_incod.info()

In [ ]:
corr_mat = players_stat_incod.corr()

In [ ]:
corr_mat

In [ ]:
# plt.figure(figsize=(40,15))
# sns.heatmap(corr_mat,annot =True)

In [ ]:
def corr_feature_detect(data,threshold=0.8):
    
    corrmat = data.corr()
    corrmat = corrmat.abs().unstack() 
    corrmat = corrmat.sort_values(ascending=False)
    corrmat = corrmat[corrmat >= threshold]
    corrmat = corrmat[corrmat < 1] 
    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']
   
    grouped_feature_ls = []
    correlated_groups = []
    
    for feature in corrmat.feature1.unique():
        if feature not in grouped_feature_ls:
    
            correlated_block = corrmat[corrmat.feature1 == feature]
            grouped_feature_ls = grouped_feature_ls + list(
                correlated_block.feature2.unique()) + [feature]
    
            correlated_groups.append(correlated_block)
    return correlated_groups

In [ ]:
corr = corr_feature_detect(data=players_stat_incod,threshold=0.1)
for i in corr:
    print(i,'\n')

In [ ]:
plt.figure(figsize=(12,8))

del corr_mat  ,players_stat
gc.collect();

### ИТОГО
- birthStateProvince - штат рождения  пропуски у иностранцев
- mlbDebutDate - дата дебюта у некоторых нет
- Возраст имеет корреляцию с Target4
-  Время карьеры имеет корреляцию в Таргет 4

### Teams

In [ ]:
teams.head()

In [ ]:
teams.info()

### ИТОГО
- Пропусков данных нет
- Явных инсайдов нет 
- возможжно как то пригодится родной стадион команды

### Seasons

In [ ]:
seasons['allStarDate'].fillna(value=pd.to_datetime('1/1/2015'), inplace=True) 
#seasons.fillna('2018-03-29')

In [ ]:
seasons_col = seasons.columns[1:]
for col in seasons_col :
    seasons[col] = pd.to_datetime(seasons[col]).values.astype('datetime64[D]')
    
#seasons['seasonStartDate'] = pd.to_datetime('seasonStartDate')

### Awards

##### Награды врученные игрокам
- есть 2 файла История до 2018  и награды в динамике (в train)


##### Идеи
- Всего наград у игрока -> влияет только на max или min  на саму динамику не влияет
- Наград в этом сизоне
- вид награды Годовая / месячная / недельная


In [ ]:
awards.head()

In [ ]:
awards.info()

In [ ]:
awards['awardSeason'].value_counts()

In [ ]:
col_obj = awards.dtypes[awards.dtypes == object].index

for col in col_obj:
    print()
    print(col)
    print(awards[col].value_counts())

# for i in col_obj :
#     print('+++++++++++++++++++++++++++++++++++++++++')
#     print(i)
#     print(i ,awards[i].unique() )

In [ ]:
time_col = ['awardDate']
for col in time_col :
    awards[col] = pd.to_datetime(awards[col]).values.astype('datetime64[D]')

In [ ]:
awards.info()

In [ ]:
awards

In [ ]:
awards['count_award'] = 1

In [ ]:
awards

In [ ]:
awards_players = awards.groupby(['playerId'], as_index = False).agg(count_awrds = ('count_award','count')).sort_values(['count_awrds'], ignore_index = True)

#  groupby(['awardId', 'awardName'], as_index = False).agg( numWinnersInThisPlayerSet = ('playerId', 'count'), mostRecAwardDate = ('awardDate', 'max'), mostRecAwardSeason = ('awardSeason', 'max')
#     ).sort_values(['mostRecAwardDate', 'awardId'],ascending = [False, True], ignore_index = True)

In [ ]:
awards_players

In [ ]:
col_aw_pl = awards_players.columns
players_stat_incod = players_stat_incod.merge(awards_players[col_aw_pl], on=['playerId'], how='left')

In [ ]:
players_stat_incod.fillna(0, inplace=True)

In [ ]:
players_stat_incod

In [ ]:
corr_mat = players_stat_incod.corr()
corr_mat

In [ ]:
corr = corr_feature_detect(data=players_stat_incod,threshold=0.1)
for i in corr:
    print(i,'\n')

In [ ]:
plt.figure(figsize=(40,15))
sns.heatmap(corr_mat,annot =True)

In [ ]:
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')

In [ ]:
awards_train = pd.read_pickle(TRAIN_DIR / 'awards_train.pkl')

In [ ]:
awards_train

In [ ]:
awards_train.info()

In [ ]:
awards_train.groupby('playerId').count()['awardId'].sort_values()[-20:].plot.barh()
plt.title('Top 20 players per number of awards for training period')

In [ ]:
top_play = awards_train.groupby('playerId').count()['awardId'].sort_values()[-20:]

In [ ]:
top_play

In [ ]:
targets.info()

In [ ]:
player_my= 665489
targets_loc = targets[targets.playerId == player_my]
targets_loc['period'] = pd.to_datetime(targets_loc['date'], format = '%Y%m%d')

awards_loc = awards_train[awards_train.playerId == player_my]
awards_loc['period'] = pd.to_datetime(awards_loc['date'], format = '%Y%m%d')
targ_col= ['playerId','target1','target2','target3','target4','date']
awards_loc = awards_loc.merge(targets_loc[targ_col], on = ['playerId','date'] , how ='left' )
awards_loc['awar_type'] = awards_loc['awardName'].apply(lambda x: x.split(' ')[-1])
awards_loc['awar_org'] = awards_loc['awardName'].apply(lambda x: x.split(' ')[0])
awards_loc

In [ ]:
for i in range(4):
    fig, ax = plt.subplots()
    ax.plot(targets_loc['period'] , targets_loc[f'target{i+1}'], color='blue')
    ax.scatter(awards_loc['period'] , awards_loc[f'target{i+1}'], color='red' )
    #aw_year = awards_loc[(awards_loc.awar_type == 'Year') & (awards_loc.awardId == 'NLROY') ]
    aw_year = awards_loc[(awards_loc.awar_org == 'AL') ]
    
    ax.scatter(aw_year['period'] , aw_year[f'target{i+1}'], color='green' )
    aw_month = awards_loc[awards_loc.awar_type == 'Month']
    ax.scatter(aw_month['period'] , aw_month[f'target{i+1}'], color='orange' )
    fig.set_figwidth(25)     #  ширина и
    fig.set_figheight(8)
    plt.legend([f'target{i+1}', 'awards' , 'year'])
    plt.title(f' target{i+1} and awards')
    plt.show()

In [ ]:
targ_col= ['playerId','target1','target2','target3','target4','date']

In [ ]:
awards_eda = awards_train.merge(targets[targ_col],on=['playerId','date'], how ='left')

In [ ]:
awards_eda

In [ ]:
awards_eda['awardName'].value_counts()

In [ ]:
awards_eda['awar_type'] = awards_eda['awardName'].apply(lambda x: x.split(' ')[-1])

In [ ]:
awards_eda

In [ ]:
awards_eda['awar_type'].value_counts()

In [ ]:
awards_type = awards_eda[['awardId','awar_type']] 

In [ ]:
awards_type = awards_type.drop_duplicates()

In [ ]:
awards_type

In [ ]:
# удаляем строки с пустрыми значениями
awards_eda =awards_eda.dropna()

In [ ]:
awards_eda

In [ ]:
df_coding_columns = ['awardId']
awards_eda_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(awards_eda)

In [ ]:
awards_eda_incod.head()

In [ ]:
awards_corr_mat = awards_eda_incod.corr()
awards_corr_mat

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(awards_corr_mat,annot =True)

In [ ]:
#### Проверка зависимости показателей от награды

In [ ]:
award_col = awards_train.columns
target_awards = targets.merge(awards_train[award_col],on=['playerId','date'], how ='left')

In [ ]:
target_awards.fillna(0, inplace=True)

In [ ]:
target_awards['awardId'].value_counts()

In [ ]:
target_awards

In [ ]:
targets_my = targets.copy() 

In [ ]:
targets_my['period'] = targets_my['date']
targets_my['period'] = pd.to_datetime(targets_my['period'], format = '%Y%m%d')
targets_my = targets_my.set_index('period').to_period('D')
#train = udate_season(train)

In [ ]:
targets_my.info()

In [ ]:
playerid = 683734

In [ ]:
data = targets_my[ (targets_my.playerId==playerid)]

In [ ]:
data

In [ ]:
data.target1

In [ ]:
del data , targets_my ,awards_eda_incod
gc.collect();

### playerTwitterFollowers

In [ ]:
playerTwitter = pd.read_pickle(TRAIN_DIR / 'playerTwitterFollowers_train.pkl')

In [ ]:
playerTwitter

In [ ]:
playerTwitter.info()

In [ ]:
playerTwitter['period'] = pd.to_datetime(playerTwitter['date'], format = '%Y%m%d')

In [ ]:
playerTwitter['twit_year'] = playerTwitter['period'].dt.year

In [ ]:
playerTwitter['twit_month'] = playerTwitter['period'].dt.month

In [ ]:
most_famous_player = playerTwitter.groupby('playerId')['numberOfFollowers'].median().sort_values(ascending=False)[:20]

In [ ]:
most_famous_player

In [ ]:
med_followers = pd.DataFrame(playerTwitter.groupby('playerId')['numberOfFollowers'].median().sort_values(ascending=False)).reset_index()
med_followers.columns = ['playerId', 'med_f']

In [ ]:
med_followers

In [ ]:
playerTwitter = playerTwitter.merge(med_followers , on =['playerId'], how = 'left')

In [ ]:
playerTwitter

In [ ]:
newColumn2 = lambda x: x['numberOfFollowers'] - x['med_f']
playerTwitter['twit_trend'] = playerTwitter.apply(newColumn2, axis=1) 

In [ ]:
playerTwitter

In [ ]:
playerTwitter['date'].value_counts()

## Train data

In [ ]:
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')
rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')
events = pd.read_pickle(TRAIN_DIR / 'events_train.pkl')

## Events

In [ ]:
events

### transactions

In [ ]:
transactions = pd.read_pickle(TRAIN_DIR / 'transactions_train.pkl')

In [ ]:
transactions

In [ ]:
trans_col = ['playerId','date','fromTeamId','toTeamId','typeCode']
trans_eda = transactions[trans_col]
trans_eda = trans_eda.merge(targets[targ_col],on=['playerId','date'], how ='left')

In [ ]:
trans_eda

In [ ]:
trans_eda['typeCode'].value_counts()

In [ ]:
trans_eda['playerId'].value_counts()

In [ ]:
player_my= 545343
targets_loc = targets[targets.playerId == player_my]
targets_loc['period'] = pd.to_datetime(targets_loc['date'], format = '%Y%m%d')

tran_loc = trans_eda[trans_eda.playerId == player_my]
tran_loc['period'] = pd.to_datetime(tran_loc['date'], format = '%Y%m%d')
tran_loc

In [ ]:
for i in range(4):
    fig, ax = plt.subplots()
    ax.plot(targets_loc['period'] , targets_loc[f'target{i+1}'], color='blue')
    ax.scatter(tran_loc['period'] , tran_loc[f'target{i+1}'], color='red' )
    #aw_year = awards_loc[(awards_loc.awar_type == 'Year') & (awards_loc.awardId == 'NLROY') ]
#     aw_year = awards_loc[(awards_loc.awar_org == 'AL') ]
    
#     ax.scatter(aw_year['period'] , aw_year[f'target{i+1}'], color='green' )
#     aw_month = awards_loc[awards_loc.awar_type == 'Month']
#     ax.scatter(aw_month['period'] , aw_month[f'target{i+1}'], color='orange' )
    fig.set_figwidth(25)     #  ширина и
    fig.set_figheight(8)
    plt.legend([f'target{i+1}', 'trans' , 'year'])
    plt.title(f' target{i+1} and trans')
    plt.show()

In [ ]:
trans_eda

In [ ]:


trans_col = ['playerId','date','fromTeamId','toTeamId','typeCode']
trans_tar = targets.merge(transactions[trans_col],on=['playerId','date'], how ='left')

#trans_eda = trans_eda[(trans_eda.typeCode == 'TR')]
df_coding_columns = ['fromTeamId' ,'toTeamId' ,'typeCode']
trans_eda_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(trans_tar)



In [ ]:
trans_corr_mat = trans_eda_incod.corr()
trans_corr_mat

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(trans_corr_mat,annot =True)

In [ ]:
del trans_corr_mat , trans_eda ,trans_eda_incod
gc.collect();

## Rosters

In [ ]:
rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')

In [ ]:
rosters

## Standings

In [ ]:
standings = pd.read_pickle(TRAIN_DIR / 'standings_train.pkl')

In [ ]:
standings.head()

In [ ]:
standings.shape

In [ ]:
standings.info()

In [ ]:
# tar_ros =  targets.merge(rosters , on = ['playerId','date'] , how ='left')
# tar_ros.head()

In [ ]:
ros_stan =  rosters.merge(standings , on = ['teamId','date'] , how ='left')
ros_stan.head()

In [ ]:
ros_stan_eda =  ros_stan.merge(targets , on = ['playerId','date'] , how ='left')
ros_stan_eda

In [ ]:
player_my= 429664
targets_loc = targets[targets.playerId == player_my]
targets_loc['period'] = pd.to_datetime(targets_loc['date'], format = '%Y%m%d')

stan_loc = ros_stan_eda[ros_stan_eda.playerId == player_my]
stan_loc['period'] = pd.to_datetime(stan_loc['date'], format = '%Y%m%d')
stan_loc

In [ ]:
# #cont_col = ['period','target1','target2','target3','target4']
# stand_col = stan_loc.columns.to_list()
# for col in stand_col:
#     cont_col = ['period','target1','target2','target3','target4']
#     cont_col.append(col)
#     print(cont_col)
#     stan_loc_col  = stan_loc[cont_col]
#     stan_loc_col = stan_loc_col.dropna()
#     for i in range(4):
#         fig, ax = plt.subplots()
#         ax.plot(targets_loc['period'] , targets_loc[f'target{i+1}'], color='blue')
#         ax.scatter(stan_loc_col['period'] , stan_loc_col[f'target{i+1}'], color='red' )
#     #aw_year = awards_loc[(awards_loc.awar_type == 'Year') & (awards_loc.awardId == 'NLROY') ]
# #     aw_year = awards_loc[(awards_loc.awar_org == 'AL') ]
    
# #     ax.scatter(aw_year['period'] , aw_year[f'target{i+1}'], color='green' )
# #     aw_month = awards_loc[awards_loc.awar_type == 'Month']
# #     ax.scatter(aw_month['period'] , aw_month[f'target{i+1}'], color='orange' )
#         fig.set_figwidth(25)     #  ширина и
#         fig.set_figheight(8)
#         plt.legend([f'target{i+1}', col ])
#         plt.title(f' target{i+1} and ')
#         plt.show()

In [ ]:
for i in range(4):
    fig, ax = plt.subplots()
    ax.plot(targets_loc['period'] , targets_loc[f'target{i+1}'], color='blue')
    ax.scatter(stan_loc['period'] , stan_loc[f'target{i+1}'], color='red' )
    #aw_year = awards_loc[(awards_loc.awar_type == 'Year') & (awards_loc.awardId == 'NLROY') ]
#     aw_year = awards_loc[(awards_loc.awar_org == 'AL') ]
    
#     ax.scatter(aw_year['period'] , aw_year[f'target{i+1}'], color='green' )
#     aw_month = awards_loc[awards_loc.awar_type == 'Month']
#     ax.scatter(aw_month['period'] , aw_month[f'target{i+1}'], color='orange' )
    fig.set_figwidth(25)     #  ширина и
    fig.set_figheight(8)
    plt.legend([f'target{i+1}', 'trans' , 'year'])
    plt.title(f' target{i+1} and trans')
    plt.show()

In [ ]:
ros_stan

In [ ]:
ros_stan.info()

In [ ]:
#trans_eda = trans_eda[(trans_eda.typeCode == 'TR')]
df_coding_columns = ['statusCode']
ros_stan_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(ros_stan)

In [ ]:
ros_stan_incod

In [ ]:
stan_col = ['playerId','date','fromTeamId','toTeamId','typeCode']
stan_tar = targets.merge(ros_stan_incod,on=['playerId','date'], how ='left')
stan_tar.fillna(0,inplace=True)
stan_tar

In [ ]:
stan_tar.info()

In [ ]:
stan_corr_mat = stan_tar.corr()
stan_corr_mat

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(stan_corr_mat,annot =True)

In [ ]:
corr = corr_feature_detect(data=stan_tar,threshold=0.05)
for i in corr:
    print(i,'\n')

In [ ]:
ros_stan.head()

In [ ]:
ros_stan_cols = ros_stan.columns.tolist()
ros_stan_cols

In [ ]:
ros_stan_cols = ros_stan.columns.tolist()
ros_stan_cols = [col for col in ros_stan_cols if col not in ['playerId', 'date', 'gameDate_x', 'teamId', 'statusCode', 'status', 'index_x', 'season','gameDate_y','teamName' ,'index_y' ,'streakCode']]


In [ ]:
ros_stan_cols

In [ ]:
col_obj = ros_stan.dtypes[ros_stan.dtypes == object].index

df_coding_columns = ['statusCode','divisionRank', 'leagueRank', 'wildCardRank',
       'leagueGamesBack', 'sportGamesBack', 'divisionGamesBack', 'pct',
       'divisionChamp', 'divisionLeader', 'wildCardLeader',
       'eliminationNumber', 'wildCardEliminationNumber']
ros_stan_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(ros_stan)

In [ ]:
ros_stan_incod.fillna(0, inplace=True )

In [ ]:
col_obj

In [ ]:
ros_stan_incod.head()

In [ ]:
ros_stan_incod['divisionLeader'].value_counts()

In [ ]:
for col in ros_stan_cols:
    print(col)
    stan_agg = ros_stan_incod.groupby('playerId')[col].agg('sum').reset_index()
    #stan_agg = ros_stan_incod.groupby('playerId')[col].agg('median').reset_index()
    stan_agg.columns = ['playerId', "#"+col]
    #targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
    targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('sum').reset_index()
    stan_agg = pd.merge(stan_agg, targets_agg, on = ['playerId'], how = 'left')
    plt.figure(figsize=(10, 2))
    plt.suptitle(f"Total {col} vs Median targets", fontsize =15)
    corr = stan_agg.drop(columns =['playerId']).corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.set(font_scale=1.4)
    plt.xticks(rotation=45)
            
    x_axis_labels = ["#"+col, 'target1','target2', 'target3', 'target4'] 

    sns.heatmap(np.array(corr["#"+col]).reshape((1,5)),
    annot = True,
    xticklabels=x_axis_labels,
    vmin=0,
    vmax=1,
    center= 0,
    cmap="RdYlGn")           
            
    plt.show()
    plt.close()

In [ ]:
stan_agg

In [ ]:
del ros_stan_incod , stan_tar , stan_agg ,ros_stan
gc.collect();

## Events

In [ ]:
events = pd.read_pickle(TRAIN_DIR / 'events_train.pkl')
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')


In [ ]:
events.head()

In [ ]:
events.info()

In [ ]:
eve_col_obj = events.dtypes[events.dtypes == object].index.tolist()
eve_col_obj

In [ ]:
for  col in eve_col_obj:
    print()
    print(col)
    print(events[col].value_counts())

## Games

In [ ]:
rosters

In [ ]:
games.head()

In [ ]:
ros_game_away = rosters.merge(games ,left_on= ['date','teamId'] , right_on= ['date','awayId'] , how ='left')
ros_game_home = rosters.merge(games ,left_on= ['date','teamId'] , right_on= ['date','homeId'] , how ='left')

In [ ]:
ros_game_away

In [ ]:
#events.head()

In [ ]:
#train_X = train[feature_cols]


ros_game_col = ['playerId','gamePk','date']
ros_game_col

In [ ]:
#ros_game_my = ros_game_away[ros_game_col]

In [ ]:
#ros_game_away

In [ ]:
df_coding_columns = ['statusCode', 'status', 'gameType',  'resumedFrom',
 'codedGameState',
 'detailedGameState',
 'isTie',
 'doubleHeader',
 'dayNight',
 'seriesDescription',
 'homeName',
 'homeAbbrev',
 'homeWinner',
 'awayName',
 'awayAbbrev',
 'awayWinner']
ros_game_away_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(ros_game_away)
#ros_game_home_incod = MultiColumnLabelEncoder(columns = df_coding_columns).fit_transform(ros_game_home)

In [ ]:
ros_game_away_incod.fillna(0, inplace=True )

In [ ]:
ros_game_away_incod.info()

In [ ]:
ros_game_away_col = ros_game_away_incod.columns.tolist()


In [ ]:
ros_game_away_ok = [col for col in ros_game_away_col if col not in ['playerId', 'date', 'gameDate_x', 'teamId', 'statusCode', 'status', 'index_x', 'season','gameDate_y','gameTimeUTC' ,'index_y' ,'resumeDate','gamePk']]


In [ ]:
for col in ros_game_away_ok:
    print(col)
    game_agg = ros_game_away_incod.groupby('playerId')[col].agg('sum').reset_index()
    #stan_agg = ros_stan_incod.groupby('playerId')[col].agg('median').reset_index()
    game_agg.columns = ['playerId', "#"+col]
    #targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
    targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('sum').reset_index()
    game_agg = pd.merge(game_agg, targets_agg, on = ['playerId'], how = 'left')
    plt.figure(figsize=(10, 2))
    plt.suptitle(f"Total {col} vs Median targets", fontsize =15)
    corr = game_agg.drop(columns =['playerId']).corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.set(font_scale=1.4)
    plt.xticks(rotation=45)
            
    x_axis_labels = ["#"+col, 'target1','target2', 'target3', 'target4'] 

    sns.heatmap(np.array(corr["#"+col]).reshape((1,5)),
    annot = True,
    xticklabels=x_axis_labels,
    vmin=0,
    vmax=1,
    center= 0,
    cmap="RdYlGn")           
            
    plt.show()
    plt.close()

In [ ]:
del  game_agg , targets_agg
gc.collect();

## events

In [ ]:
ros_game_my = ros_game_away[ros_game_col]
del  ros_game_away
gc.collect();

In [ ]:
ros_game_my.head()

In [ ]:
events.head()

In [ ]:
ros_game_even = ros_game_my.merge(events , on  =['gamePk','date'], how = 'left')